In [4]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        
    }}
);

<IPython.core.display.Javascript object>

link of data:[URA website](https://www.ura.gov.sg/realEstateIIWeb/transaction/submitSearch.action)

# Threading the property Market in singapore.
## Rent or buy? 
Given a situation where an adult is given the option to buy, assuming he has sufficient for a down payment, should he or she go on to buy a property? Is property truly an asset?

Here, we will consider a specific private apartment, ParkView situated in Singapore, with relatively low variables, (no launch of MRT), to fully capture the depreciation of property as an asset, ceteris paribus. The analysis will span a period of 20 years, since its launch in 1998.


## Defining the problem/ Scoping
To do an anlysis of which is better, it is important that we define the factors we will need, the variables, to make an accurate analysis. Hence, i will be laying them down here.
They consist of:
    1. Transaction Prices
    2. CPI
    3. Interest Rates
    4. Rental

In [5]:
#import files
import pandas as pd
import seaborn as sns
import numpy as np
import sys
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='s9600820a', api_key='5CkH8mTNtfDR2N6LZvmG')

In [6]:
CPIdf = pd.read_csv("CPI123.csv")
ax = sns.barplot(x="year", y="value", data=CPIdf)
plt.show()

FileNotFoundError: File b'CPI123.csv' does not exist

The above graph is the Consumer price index over the years, pegged to 1998. Next, we look at the rental for the past 20 years. Unfortunately, I was only able to get my hands on the average over the span of 20 years, and not individual data.<br/> <img src="files/rent.jpg">

In [ ]:
rentPerMonthPSM = 21.7

Now we will be looking at the raw Data we have.(Tranactions in ParkView)

In [ ]:
RawPVdf = pd.read_csv("RawparkviewTransactions.csv")
print(list(RawPVdf))# headers
RawPVdf=RawPVdf.drop(['Address','No. of Units'],axis =1)
RawPVdf.iloc[np.r_[0:2, -2:0]]

As you can see, the data contains accurate transactions, but we will be removing majority of the columns, keeping price, getting the average transaction price per year, and adjusting with CPI. 

In [ ]:
PVdf=RawPVdf.loc[:, ['Unit Price ($ psm)','Sale Date','Purchaser Address Indicator' ]]
PVdf.iloc[np.r_[0:2, -2:0]]

In [ ]:
#PVdf['year']=PVdf['Sale Date'].dt.strftime('Y')
PVdf['year']=PVdf['Sale Date'].str.slice(-2)

In [ ]:
#create pivot ttable just for fun
impute_grps = PVdf.pivot_table(index='year', columns='Purchaser Address Indicator', aggfunc='count', fill_value=0)
impute_grps["Sale Date"]

this is just for my curiosity on the demographics of people who purchase property.<br/> Back to work

In [ ]:
def completeDate(year):
    year=int(year)
    if year<50:
        return 2000+year
    else:
        return 1900+year

In [ ]:
PVdf['year']=PVdf.year.apply(completeDate)
PVdf=PVdf.drop(['Sale Date','Purchaser Address Indicator'],axis =1)
PVdf.iloc[np.r_[0:2, -2:0]]

In [ ]:

PVdf.columns = ['UnitPrice','year']
PVdf=PVdf.groupby(['year']).UnitPrice.agg(['count','mean'])
PVdf=PVdf.reset_index()


So here, i faced an issue which took sometime to fix. What happend was that there were 2018 transactions, but not infaltion for 2018. Hence, i had to take out 2018 transactions. Or asssume zero inflation.

In [ ]:
var = CPIdf.at[19,'value']
var
CPIdf.append(pd.Series([0,2018,var], index=CPIdf.columns) , ignore_index=True)
CPIdf = CPIdf.drop(['Unnamed: 0'],axis =1)
PVdf =pd.merge(PVdf, CPIdf,on='year')
PVdf

Quick breakdown on the progress, merged the dataframes, and added count of transactions just for extras. We see that there is a spike in transactions during 2007, 2008. Now we will have to adjust the price to inflation pegged to 1998.

In [ ]:
PVdf['mean_adjusted'] = PVdf['mean']/PVdf['value']
PVdf.iloc[np.r_[0:2, -2:0]]

In [ ]:
#ax = sns.lineplot(PVdf.year,PVdf.mean_adjusted)
#plt.show()
# trace = go.Scatter(
#     x = PVdf.mean_adjusted,
#     y = PVdf.year,
#     mode = 'lines+markers',
#     name = 'lines+markers'
# )

# data = [trace]
# #plotly.offline.plot(data, filename='line-mode.html')
# plotly.offline.iplot(data, filename='line-mode')
#iplot(x=[1,2,3],y=[3,4,5])])
N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N)+5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N)-5

trace =  go.Scatter(
    x =  PVdf.year,
    y = PVdf.mean_adjusted,
    mode = 'lines+markers',
    name = 'lines+markers',
  
)
layout = go.Layout(
    title='Property Appreciation',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title=' Mean PSM price($)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

data = [trace]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='propertyApp')

So here we even in the case of a property with little development in the region, price of property is able to maintain its value.<br/><br/> <br/> <img src="calc.png"><br/>



Hence, to calculate the cost between the two. For Renting, we will plot a linear cost of 21.70(PSM) for 20 years, while for owning, will be a linear cost with $5103 / (12 X 20) for simplicity, with a slight offset in the appreciation of the property from 1998 to 2017. We will be assuming 4% interest.

In [ ]:
propertyAppreciation =(PVdf.at[19, 'mean_adjusted']-PVdf.at[0, 'mean_adjusted'])/20
interestPeryear=2318/20
salePropertyCostlist=[]
rentCostlist=[]
for i in range(1,21):
    rentCostlist.append(rentPerMonthPSM*i*12)
    salePropertyCostlist.append((interestPeryear-propertyAppreciation)*i)

traceRent =  go.Scatter(
    x =  PVdf.year,
    y = rentCostlist,
    mode = 'lines+markers',
    name = 'Renting',
  
)

traceSale =  go.Scatter(
    x =  PVdf.year,
    y = salePropertyCostlist,
    mode = 'lines+markers',
    name = 'Owning',
  
)
layout = go.Layout(
    title='Cost of Owning vs Renting',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title=' cost ($)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

data1 = [traceRent,traceSale]
fig1 = go.Figure(data=data1, layout=layout)

py.iplot(fig1, filename='OwnVSRent')

Remember, this is the cost PSM. Assuming that you bought a property of 100 Square meter, it would be a hundred fold. Leaving you in a deficit of over a $100,000.  